In [94]:
! pip install ta

In [95]:
# For prepare data
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import ta
from ta import add_all_ta_features
from ta.momentum import RSIIndicator, StochasticOscillator, WilliamsRIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import MACD, ADXIndicator

from tqdm import tqdm

# For modeling
import math
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torch import optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
# from pytorch_optimizer import SAM

# Prepare data

In [96]:
# 1. Hàm load data
def load_data(file_path):
    df = pd.read_csv(file_path)
    # Xử lý datetime
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df = df.sort_values('Datetime').drop(['Date', 'Time'], axis=1)
    return df

train_df = load_data("/kaggle/input/xauusd1m/dynamic_labeled_train.csv")
val_df = load_data("/kaggle/input/xauusd1m/dynamic_labeled_dev.csv")
test_df = load_data("/kaggle/input/xauusd1m/dynamic_labeled_test.csv")

# train_df = train_df.loc[train_df['Datetime'].dt.year.isin(range(2018, 2021))]

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3446549 entries, 0 to 3446548
Data columns (total 7 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Open      float64       
 1   High      float64       
 2   Low       float64       
 3   Close     float64       
 4   Volume    int64         
 5   Label     object        
 6   Datetime  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 184.1+ MB


In [97]:
label_mapping = {
    'BUY': 0,
    'SELL': 1,
    'HOLD': 2
}

def map_label(x):
    return label_mapping[x] if x in label_mapping else x

train_df['Label'] = train_df['Label'].map(map_label)
val_df['Label'] = val_df['Label'].map(map_label)
test_df['Label'] = test_df['Label'].map(map_label)

In [98]:
def add_technical_indicators(df):
    """Thêm các chỉ báo kỹ thuật."""
    print("Thêm các chỉ báo kỹ thuật...")
    # Momentum
    df['RSI'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    df['Momentum'] = ta.momentum.ROCIndicator(df['Close']).roc()
    df['CMO'] = ta.momentum.kama(df['Close'])
    df['Williams_%R'] = ta.momentum.WilliamsRIndicator(df['High'], df['Low'], df['Close']).williams_r()
    # Volatility
    df['ATR'] = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close']).average_true_range()
    bb = ta.volatility.BollingerBands(df['Close'])
    df['BB_Mid'] = bb.bollinger_mavg()
    df['BB_Upper'] = bb.bollinger_hband()
    df['BB_Lower'] = bb.bollinger_lband()
    df['BB_Bandwidth'] = bb.bollinger_wband()
    keltner = ta.volatility.KeltnerChannel(df['High'], df['Low'], df['Close'])
    df['KC_High'] = keltner.keltner_channel_hband()
    df['KC_Low'] = keltner.keltner_channel_lband()
    donchian = ta.volatility.DonchianChannel(df['High'], df['Low'], df['Close'])
    df['DC_High'] = donchian.donchian_channel_hband()
    df['DC_Low'] = donchian.donchian_channel_lband()
    # Trend
    df['SMA_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['EMA_20'] = ta.trend.EMAIndicator(df['Close'], window=20).ema_indicator()
    df['DPO'] = ta.trend.DPOIndicator(df['Close']).dpo()
    macd = ta.trend.MACD(df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Hist'] = macd.macd_diff()
    df['Mass_Index'] = ta.trend.mass_index(df['High'], df['Low'])
    # Volume
    df['AD'] = ta.volume.AccDistIndexIndicator(df['High'], df['Low'], df['Close'], df['Volume']).acc_dist_index()
    df['CMF'] = ta.volume.ChaikinMoneyFlowIndicator(df['High'], df['Low'], df['Close'], df['Volume']).chaikin_money_flow()
    df['Force_Index'] = ta.volume.ForceIndexIndicator(df['Close'], df['Volume']).force_index()
    df['MFI'] = ta.volume.MFIIndicator(df['High'], df['Low'], df['Close'], df['Volume']).money_flow_index()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['Close'], df['Volume']).on_balance_volume()

    print("Hoàn thành thêm chỉ báo.")
    return df.reset_index(drop=True)

train_df = add_technical_indicators(train_df)
val_df = add_technical_indicators(val_df)
test_df = add_technical_indicators(test_df)

train_df.info()

Thêm các chỉ báo kỹ thuật...
Hoàn thành thêm chỉ báo.
Thêm các chỉ báo kỹ thuật...


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


Hoàn thành thêm chỉ báo.
Thêm các chỉ báo kỹ thuật...
Hoàn thành thêm chỉ báo.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3446549 entries, 0 to 3446548
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Open          float64       
 1   High          float64       
 2   Low           float64       
 3   Close         float64       
 4   Volume        int64         
 5   Label         int64         
 6   Datetime      datetime64[ns]
 7   RSI           float64       
 8   Momentum      float64       
 9   CMO           float64       
 10  Williams_%R   float64       
 11  ATR           float64       
 12  BB_Mid        float64       
 13  BB_Upper      float64       
 14  BB_Lower      float64       
 15  BB_Bandwidth  float64       
 16  KC_High       float64       
 17  KC_Low        float64       
 18  DC_High       float64       
 19  DC_Low        float64       
 20  SMA_20        float64       
 21  EMA_20        float64

In [99]:
def add_basic_features(df):
    """Tiền xử lý cuối: pct_change, ánh xạ nhãn, thêm đặc trưng thời gian."""
    print("Áp dụng tiền xử lý...")
    # Tính phần trăm thay đổi cho OHLC
    cols_to_pct = ['Open', 'High', 'Low', 'Close']
    existing_cols = [col for col in cols_to_pct if col in df.columns]
    if existing_cols:
        print(f"Tính phần trăm thay đổi cho: {existing_cols}")
        df[existing_cols] = df[existing_cols].pct_change().fillna(0) * 100
        df['Cum_Return'] = df['Close'].rolling(window=20).sum()
        df['Cum_Turnover'] = df['Volume'].rolling(window=20).sum()
    else:
        print("Cảnh báo: Không tìm thấy cột OHLC để tính pct_change.")

    # Thêm đặc trưng thời gian
    if 'Datetime' in df.columns:
        print("Thêm đặc trưng thời gian (hour, day_of_week, minute_of_day, index)...")
        df['Hour'] = df['Datetime'].dt.hour / 23.0
        df['Day_Of_Week'] = df['Datetime'].dt.dayofweek / 6.0
        df['Minute_Of_Day'] = (df['Datetime'].dt.hour * 60 + df['Datetime'].dt.minute) / 1439.0
        # df = df.drop(columns=['Datetime'], errors='ignore')
    else:
        print("Cảnh báo: Không thể tạo đặc trưng thời gian do thiếu cột 'Datetime'.")

    print("Hoàn thành tiền xử lý.")
    return df.reset_index(drop=True)

train_df = add_basic_features(train_df)
val_df = add_basic_features(val_df)
test_df = add_basic_features(test_df)
train_df.info()

Áp dụng tiền xử lý...
Tính phần trăm thay đổi cho: ['Open', 'High', 'Low', 'Close']
Thêm đặc trưng thời gian (hour, day_of_week, minute_of_day, index)...
Hoàn thành tiền xử lý.
Áp dụng tiền xử lý...
Tính phần trăm thay đổi cho: ['Open', 'High', 'Low', 'Close']
Thêm đặc trưng thời gian (hour, day_of_week, minute_of_day, index)...
Hoàn thành tiền xử lý.
Áp dụng tiền xử lý...
Tính phần trăm thay đổi cho: ['Open', 'High', 'Low', 'Close']
Thêm đặc trưng thời gian (hour, day_of_week, minute_of_day, index)...
Hoàn thành tiền xử lý.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3446549 entries, 0 to 3446548
Data columns (total 36 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Open           float64       
 1   High           float64       
 2   Low            float64       
 3   Close          float64       
 4   Volume         int64         
 5   Label          int64         
 6   Datetime       datetime64[ns]
 7   RSI            float64       
 8   M

In [100]:
def drop_na_cols(df, threshold=0.01):
    cnt = 0
    for col in df.columns:
        na = df[[col]].isna().sum()
        if na.values > len(df) * threshold:
            df.drop(col, axis=1, inplace=True)
            cnt += 1
    print(f'Deleted {cnt} cols')
    return df

# 6. Hàm xử lý missing values
def handle_missing_data(df, threshold=0.01):
    df = drop_na_cols(df, threshold)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    df = df.bfill().ffill()
    df = df.dropna()
    
    return df

# train_df
train_df = handle_missing_data(train_df)
val_df = handle_missing_data(val_df)
test_df = handle_missing_data(test_df)
train_df.info()

Deleted 0 cols
Deleted 0 cols
Deleted 0 cols
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3446549 entries, 0 to 3446548
Data columns (total 36 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Open           float64       
 1   High           float64       
 2   Low            float64       
 3   Close          float64       
 4   Volume         int64         
 5   Label          int64         
 6   Datetime       datetime64[ns]
 7   RSI            float64       
 8   Momentum       float64       
 9   CMO            float64       
 10  Williams_%R    float64       
 11  ATR            float64       
 12  BB_Mid         float64       
 13  BB_Upper       float64       
 14  BB_Lower       float64       
 15  BB_Bandwidth   float64       
 16  KC_High        float64       
 17  KC_Low         float64       
 18  DC_High        float64       
 19  DC_Low         float64       
 20  SMA_20         float64       
 21  EMA_20         float64       
 2

In [101]:
def normalize_by_blocks(data, block_size):
    print(f"Áp dụng normalize_by_blocks với block_size={block_size}...")
    if isinstance(data, pd.DataFrame):
        columns = data.columns
        index = data.index
        data_np = data.values.astype('float32')
    else:
        data_np = data.astype('float32')
        columns = None
        index = None

    result = np.zeros_like(data_np)
    num_blocks = 0

    for start_idx in range(0, len(data_np), block_size):
        end_idx = min(start_idx + block_size, len(data_np))
        block = data_np[start_idx:end_idx]

        if block.shape[0] > 0:
            scaler = StandardScaler()
            if block.shape[0] == 1:
                normalized_block = block - np.mean(block, axis=0)
            else:
                std_devs = np.std(block, axis=0)
                if np.any(std_devs == 0):
                    normalized_block = np.zeros_like(block)
                    valid_cols = std_devs != 0
                    if np.any(valid_cols):
                        scaler.fit(block[:, valid_cols])
                        normalized_block[:, valid_cols] = scaler.transform(block[:, valid_cols])
                    zero_std_cols = std_devs == 0
                    if np.any(zero_std_cols):
                        normalized_block[:, zero_std_cols] = block[:, zero_std_cols] - np.mean(block[:, zero_std_cols], axis=0)
                else:
                    normalized_block = scaler.fit_transform(block)

            if np.isnan(normalized_block).any() or np.isinf(normalized_block).any():
                normalized_block = np.nan_to_num(normalized_block, nan=0.0, posinf=0.0, neginf=0.0)

            result[start_idx:end_idx] = normalized_block
            num_blocks += 1

    print(f"Hoàn thành normalize_by_blocks. Đã xử lý {num_blocks} khối.")
    if columns is not None and index is not None:
        return pd.DataFrame(result, columns=columns, index=index)
    else:
        return result

# Định nghĩa các cột đặc trưng
feature_cols = [col for col in train_df.columns if col not in ['Label', 'Datetime']]
print(f"Sử dụng {len(feature_cols)} cột đặc trưng: {feature_cols}")

# Áp dụng chuẩn hóa theo khối
BLOCK_SIZE = 128
train_df[feature_cols] = normalize_by_blocks(train_df[feature_cols], BLOCK_SIZE)
val_df[feature_cols] = normalize_by_blocks(val_df[feature_cols], BLOCK_SIZE)
test_df[feature_cols] = normalize_by_blocks(test_df[feature_cols], BLOCK_SIZE)



Sử dụng 34 cột đặc trưng: ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'Momentum', 'CMO', 'Williams_%R', 'ATR', 'BB_Mid', 'BB_Upper', 'BB_Lower', 'BB_Bandwidth', 'KC_High', 'KC_Low', 'DC_High', 'DC_Low', 'SMA_20', 'EMA_20', 'DPO', 'MACD', 'MACD_Hist', 'Mass_Index', 'AD', 'CMF', 'Force_Index', 'MFI', 'OBV', 'Cum_Return', 'Cum_Turnover', 'Hour', 'Day_Of_Week', 'Minute_Of_Day']
Áp dụng normalize_by_blocks với block_size=128...
Hoàn thành normalize_by_blocks. Đã xử lý 26927 khối.
Áp dụng normalize_by_blocks với block_size=128...
Hoàn thành normalize_by_blocks. Đã xử lý 2743 khối.
Áp dụng normalize_by_blocks với block_size=128...
Hoàn thành normalize_by_blocks. Đã xử lý 8808 khối.


In [102]:
train_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Label', 'Datetime', 'RSI',
       'Momentum', 'CMO', 'Williams_%R', 'ATR', 'BB_Mid', 'BB_Upper',
       'BB_Lower', 'BB_Bandwidth', 'KC_High', 'KC_Low', 'DC_High', 'DC_Low',
       'SMA_20', 'EMA_20', 'DPO', 'MACD', 'MACD_Hist', 'Mass_Index', 'AD',
       'CMF', 'Force_Index', 'MFI', 'OBV', 'Cum_Return', 'Cum_Turnover',
       'Hour', 'Day_Of_Week', 'Minute_Of_Day'],
      dtype='object')

In [103]:
class PreprocessedStockDataset(Dataset):
    def __init__(self, df, sequence_length, feature_cols, use_time2vec=True):
        self.sequence_length = sequence_length
        self.feature_cols = [col for col in feature_cols if col in df.columns]
        self.use_time2vec = use_time2vec
        self.has_time_input = False

        print(f"Khởi tạo PreprocessedStockDataset với {len(self.feature_cols)} đặc trưng.")
        if self.use_time2vec:
            if "Minute_Of_Day" in df.columns and pd.api.types.is_numeric_dtype(df['Minute_Of_Day']):
                print("Sử dụng 'minute_of_day' cho Time2Vec.")
                self.time_features = df["Minute_Of_Day"].values[:, np.newaxis].astype(np.float32)
                self.has_time_input = True
            else:
                print("Cảnh báo: Không tìm thấy 'minute_of_day'. Time2Vec sẽ dùng giá trị 0.")
                self.time_features = np.zeros((len(df), 1), dtype=np.float32)
        else:
            self.time_features = np.zeros((len(df), 1), dtype=np.float32)

        self.features = df[self.feature_cols].values.astype(np.float32)
        if 'Label' in df.columns and pd.api.types.is_numeric_dtype(df['Label']):
            self.labels = df['Label'].values.astype(np.int64)
            print(f"Kích thước - Features: {self.features.shape}, Labels: {self.labels.shape}, Time: {self.time_features.shape}")
        else:
            print("Cảnh báo: Không tìm thấy cột 'Label'. Tạo nhãn giả (0).")
            self.labels = np.zeros(len(df), dtype=np.int64)

        if len(self.features) <= self.sequence_length:
            raise ValueError(f"Độ dài DataFrame ({len(self.features)}) phải lớn hơn sequence_length ({self.sequence_length}).")

    def __len__(self):
        return len(self.features) - self.sequence_length

    def __getitem__(self, idx):
        end_idx = idx + self.sequence_length
        feat_seq = self.features[idx:end_idx]
        time_seq = self.time_features[idx:end_idx]
        label = self.labels[end_idx]
        return torch.from_numpy(feat_seq).float(), torch.from_numpy(time_seq).float(), torch.tensor(label).long()

# Tạo Dataset
USE_TIME2VEC = True
SEQ_LEN = 128
print("\n--- Tạo Dataset và DataLoader ---")
train_dataset = PreprocessedStockDataset(train_df, SEQ_LEN, feature_cols, use_time2vec=USE_TIME2VEC)
val_dataset = PreprocessedStockDataset(val_df, SEQ_LEN, feature_cols, use_time2vec=USE_TIME2VEC)
test_dataset = PreprocessedStockDataset(test_df, SEQ_LEN, feature_cols, use_time2vec=USE_TIME2VEC)

# Tạo DataLoader
BATCH_SIZE = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=torch.cuda.is_available())
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=2, pin_memory=torch.cuda.is_available())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=2, pin_memory=torch.cuda.is_available())


--- Tạo Dataset và DataLoader ---
Khởi tạo PreprocessedStockDataset với 34 đặc trưng.
Sử dụng 'minute_of_day' cho Time2Vec.
Kích thước - Features: (3446549, 34), Labels: (3446549,), Time: (3446549, 1)
Khởi tạo PreprocessedStockDataset với 34 đặc trưng.
Sử dụng 'minute_of_day' cho Time2Vec.
Kích thước - Features: (350984, 34), Labels: (350984,), Time: (350984, 1)
Khởi tạo PreprocessedStockDataset với 34 đặc trưng.
Sử dụng 'minute_of_day' cho Time2Vec.
Kích thước - Features: (1127367, 34), Labels: (1127367,), Time: (1127367, 1)


In [104]:
# Kiểm tra
print("Number of batches:", len(train_loader))
sample_batch = next(iter(train_loader))
print("Batch input shape:", sample_batch[0].shape)
print("Batch times shape:", sample_batch[1].shape)
print("Batch labels shape:", sample_batch[2].shape)
print("\nExample input shape for Transformer:", sample_batch[0][0].shape)
print(sample_batch[0][0])

Number of batches: 13463
Batch input shape: torch.Size([256, 128, 34])
Batch times shape: torch.Size([256, 128, 1])
Batch labels shape: torch.Size([256])

Example input shape for Transformer: torch.Size([128, 34])
tensor([[ 0.6224,  0.8578,  0.7255,  ...,  0.9812,  0.0000,  0.7443],
        [ 0.5433,  0.2644, -0.0107,  ...,  0.9812,  0.0000,  0.7713],
        [-0.3728, -0.6069, -0.3159,  ...,  0.9812,  0.0000,  0.7984],
        ...,
        [-0.9408, -0.0384, -0.0298,  ...,  0.6713,  0.0000,  0.6631],
        [ 0.5465, -0.0837,  0.2351,  ...,  0.6713,  0.0000,  0.6901],
        [-0.2076, -0.5815, -2.3689,  ...,  0.6713,  0.0000,  0.7172]])


# Model

In [ ]:
class InceptionModule(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.branch1 = nn.Conv1d(in_channels, 32, kernel_size=1, padding='same')
        self.branch3 = nn.Conv1d(in_channels, 32, kernel_size=3, padding='same')
        self.branch5 = nn.Conv1d(in_channels, 32, kernel_size=5, padding='same')
        self.branch_pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, 32, kernel_size=1)
        )

    def forward(self, x):
        return torch.cat([self.branch1(x), self.branch3(x), self.branch5(x), self.branch_pool(x)], dim=1)

class Time2Vec(nn.Module):
    def __init__(self, time_dim, kernel_dim=32):
        super().__init__()
        self.linear = nn.Linear(time_dim, 1)
        self.periodic = nn.Linear(time_dim, kernel_dim - 1)

    def forward(self, x):
        linear = self.linear(x)
        periodic = self.periodic(x)
        return torch.cat([linear, periodic], dim=-1)

class CrossAttentionFusion(nn.Module):
    def __init__(self, cnn_dim, transformer_dim):
        super().__init__()
        self.query = nn.Linear(cnn_dim, transformer_dim)
        self.key = nn.Linear(transformer_dim, transformer_dim)
        self.value = nn.Linear(transformer_dim, transformer_dim)
        
    def forward(self, cnn_features, transformer_features):
        Q = self.query(cnn_features).unsqueeze(1)  # [batch, 1, transformer_dim]
        K = self.key(transformer_features)         # [batch, seq_len, transformer_dim]
        V = self.value(transformer_features)       # [batch, seq_len, transformer_dim]
        
        attn_scores = (Q @ K.transpose(-2, -1)) / (K.size(-1) ** 0.5)  # [batch, 1, seq_len]
        attn_weights = torch.softmax(attn_scores, dim=-1)
        
        return torch.bmm(attn_weights, V).squeeze(1)  # [batch, transformer_dim]

class HighwayNetwork(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.gate = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Sigmoid()
        )
        
    def forward(self, fused, transformer):
        g = self.gate(fused)
        return g * fused + (1 - g) * transformer

class EnhancedHybridModel(nn.Module):
    def __init__(self, num_features, time_dim, num_classes=3, d_model=512, nhead=16, dim_feedforward=1024, num_layers=6):
        super().__init__()
        # 1. InceptionTime Branch
        self.inception = nn.Sequential(
            InceptionModule(num_features),
            nn.ReLU(),
            nn.MaxPool1d(2),
            InceptionModule(128),
            nn.ReLU()
        )
        
        # 2. Transformer Branch
        self.time2vec = Time2Vec(time_dim, d_model//2)
        self.transformer_proj = nn.Linear(num_features, d_model//2)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # 3. Fusion
        self.cross_attention = CrossAttentionFusion(128, d_model)
        self.highway = HighwayNetwork(d_model)
        
        # 4. Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, time_feature):
        # 1. Inception Path
        cnn_features = self.inception(x.permute(0, 2, 1))  # [batch, channels, seq_len//2]
        cnn_features = cnn_features.mean(dim=-1)          # [batch, channels=128]
        
        # 2. Transformer Path
        time_embed = self.time2vec(time_feature)  # [batch, seq_len, d_model // 2]
        x_proj = self.transformer_proj(x)  # [batch, seq_len, d_model // 2]
        combined = torch.cat([time_embed, x_proj], dim=-1) # [batch, seq_len, d_model]
        transformer_features = self.transformer(combined)  # [batch, seq_len, d_model]
        
        # 3. Fusion
        fused = self.cross_attention(cnn_features, transformer_features)  # [batch, d_model]
        output = self.highway(fused, transformer_features.mean(dim=1))   # [batch, d_model]
        
        return self.classifier(output)

N_FEATURES = sample_batch[0].shape[-1]
model = EnhancedHybridModel(
    num_features=N_FEATURES, time_dim=1, num_classes=3, 
    d_model=128, nhead=8, dim_feedforward=256, num_layers=3
)

model.load_state_dict(torch.load('save/models/used_model.pth', weights_only=True, map_location='cpu'))